In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# ****Trabalho - Python II****

# **Aluna: Vládia Barreira Beserra**
    
## Análise de Dados sobre base com informações relevantes na indicação de indivíduos com e sem Alzheimer

A base utilizada neste estudo contempla dados de ressonância magnética do cérebro e foi disponibilizada para estudo pelo canal aberto de estudos de imagem (OASIS). O OASIS é disponibilizado pelo Centro de Pesquisa em Doença de Alzheimer da Universidade de Washington, pelo Dr. Randy Buckner no Instituto Médico Howard Hughes (HHMI) (na Universidade de Harvard, no Grupo de Pesquisa em Neuroinformática (NRG) da Escola de Medicina da Universidade de Washington e pela Pesquisa em Informática Biomédica Rede (BIRN).

Este conjunto consiste em 436 indivíduos de 18 a 96 anos. Para cada sujeito, 3 ou 4 exames de ressonância magnética individuais foram realizados. Os sujeitos são todos destros e incluem homens e mulheres. 100 dos indivíduos incluídos acima de 60 anos foram diagnosticados clinicamente com doença de Alzheimer muito leve a moderada (DA). 

In [ ]:
# Obtendo dados a serem analisados
ocs = pd.read_csv('../input/mri-and-alzheimers/oasis_cross-sectional.csv')

### Analisaremos primeiro os dados transversais de ressonância magnética, através das informações contidas no arquivo oasis_cross_sectional

* ID - Identificação do indivíduo
* M/F - Gênero
* Hand - Mão dominante (Destro/Canhoto/Bidestro)
* Age - Idade
* Educ - Nível Educacioanal
* SES - Status Sócio Econômico
* MMSE - Estado Mental - Mini Mental State Examination 
* (comumente usado como parte do processo diagnóstico de demência, de acordo com a definição do Diagnostic and Statistical Manual of Mental Disorders (DSM). 
* O MMSE é um questionário escrito que tem um escore máximo de 30 pontos. Escores mais baixos indicam problemas cognitivos mais graves. 
* O ponto de corte do MMSE que indica uma função cognitiva “normal” é de 24 pontos. Porém, teoricamente, o escore pode ficar entre 1 e 30 pontos. )
* CDR - Classificação de Demência
* eTIV - Volume Estimado Intracraneado 
* nWBV - Volume Total do Cerébro
* ASF - Atlas Scaling Factor - Fator de escala do Atlas - Matriz de transformação que determina o volume do cérebro.
* Delay 


In [ ]:
# Visualizando os dados 
ocs.head(5)

In [ ]:
# Compreendendo os quantitativos gerais e tipos de dados.
ocs.info()

## Descrição dos Dados

A partir da descrição geral dos dados abaixo, podemos notar as seguintes informações:
* Foram analisadas pessoas entre 18 e 96 anos;
* O nível de educação pode variar entre 1 e 5, assim como o Status Sócio Econômico;
* O estado mental dos indivíduos relativo à indicação de demência, mensurado pelo teste neuropsicológico MMSE, variaram entre 14 e 30, apresentando média de 27. Observando que o ponto de corte deste exame é 24, considera-se as pessoas com pontuação abaixo deste valor com certo grau de acomentimento mental. Dessa forma, podemos dizer que esta média é otimista.

In [ ]:
# Compreendendo as pricipais medidas, assim como as suas distribuições totais.
ocs.describe()

Agora que temos algumas medidas e distribuições, vamos explorar algumas informações específicas para termos algumas inferências.

Na tentativa de compreender como estava sendo a distribuição da idade dos dados analisados, visto que esta informação faria parte de um dos parâmetros de análise, foram emitidos dois gráficos, com a ideia de um validar o comportamento do outro.
Um histograma e outro com indicação de distribuições máximas. 

In [ ]:
# Usando o seaborn
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
ocs['Age'].hist(bins=40)

In [ ]:
sns.distplot(ocs['Age'].dropna())

Obtém-se assim a presença maior de indivíduos entre 20 e 25 anos, e 70 e 80 anos.

In [ ]:
# Renomeando a coluna 'M/F' para 'Genero'
ocs.rename(columns={'M/F': 'Genero'}, inplace=True)

Diante dos quantitativos abaixo, concluímos que a análise foi feita predominantemente em pessoas de gênero do sexo feminino.

In [ ]:
# A análise foi feita predominantemente em pessoas de gênero do sexo feminino.
ocs['Genero'].value_counts()

No gráfico abaixo podemos inferir que, embora haja similaridade nos índices de demências entre homens e mulheres, percebe-se que, quando o índice está menor, abaixo de 24, sua ocorrência maior é nos homens.

In [ ]:
# Vamos entender em qual gênero há maior incidência de demência.
sns.distplot(ocs[ocs['Genero'] == 'F']['MMSE'].dropna(), label='Feminino')
sns.distplot(ocs[ocs['Genero'] == 'M']['MMSE'].dropna(), label='Masculino')
plt.legend()

Com o objetivo de possuir mais itens que validassem as informações do gráfico acima, obteve-se a média do MMSE de homens e mulheres e a disposição dessa informação em gráfico de distribuição boxplot. Os homens obtiveram média de valor do MMSE abaixo das mulheres, assim como também indicado no referido gráfico.

In [ ]:
# Média dos valores de MMSE entre homens e mulheres.
ocs.groupby('Genero')['MMSE'].mean()

In [ ]:
sns.boxplot(x='Genero', y='MMSE', data=ocs)

Quanto ao volume intracraneano, medido pelo atributo eTIV, é possível observar no gráfico abaixo que o sexo feminino possui volume mais reduzido que o sexo masculino. Esta diferença também pode ser percebida pelo índice de Volume Total do Cérebro, medido pelo nWBV.

In [ ]:
# eTIV - Volume Estimado Intracraneano
sns.distplot(ocs[ocs['Genero'] == 'F']['eTIV'].dropna(), label='Feminino')
sns.distplot(ocs[ocs['Genero'] == 'M']['eTIV'].dropna(), label='Masculino')
plt.legend()

De forma a corroborar estes valores de forma numérica e precisa, medimos abaixo a média do índice eTIV entre homens e mulheres. Observa-se assim, a confirmação do volume intracraneano menor em mulheres.

In [ ]:
# Media do índice de Volume Intracraneano agrupadas por Gênero
ocs.groupby('Genero')['eTIV'].mean()

In [ ]:
# nWBV - Volume Total do Cérebro
sns.distplot(ocs[ocs['Genero'] == 'F']['nWBV'].dropna(), label='Feminino')
sns.distplot(ocs[ocs['Genero'] == 'M']['nWBV'].dropna(), label='Masculino')
plt.legend()

Já o Volume Total do Cérebro recebe menos variação que o Volume Intracraneano. Essa informação pode ser percebida tanto no gráfica logo acima, quanto nas médias expostas abaixo.

In [ ]:
# Média dos valores de nWBV entre homens e mulheres.
ocs.groupby('Genero')['nWBV'].mean()

Para mesclar as análises realizadas anteriormente sobre volumes relativos à índices craneanos, foi elaborado um gráfico de dispersão de forma a compreender visualmente a relação entre as variáveis eTIV e nWBR análisadas sobre o aspecto de gênero. Conforme visualização abaixo, podemos perceber menor variabilidade no índice de nWBV e maior no índice eTIV. 

In [ ]:
sns.pairplot(x_vars='nWBV' ,y_vars='eTIV' ,data=ocs , height=5, hue='Genero')

### **Idade x Volume Total do Cérebro (nWBV)**

E quanto à idade? Será verdade que quanto mais a idade avança, tem-se diminuição do volume total do cérebro?

Abaixo temos um gráfico de barras que indica sim, que quanto mais velho ficamos, o volume total do cérebro dimuniu.

In [ ]:
sns.barplot(x='nWBV' ,y='Age' ,data=ocs)

Antes de emitir o gráfico abaixo, verificou-se que o Volume Intracraneano não se altera de forma significativa ao longo dos anos. Optou-se então, verificar a relação do Volume Total do Cérebro (nWBV) com Teste Neuropsicológico (MMSE) relacionado à idade.

Notou-se assim, que ao decorrer da idade há uma relação inversa entre essas duas variáveis, ao passo que enquanto o volume diminui, a incidência de alterações cognitivas aumentam.

In [ ]:

sns.pairplot(x_vars='nWBV' ,y_vars='MMSE' ,data=ocs , height=5, hue='Age')

### **A demência**

Não existe um exame específico pra fazer o diagnóstico desta doença. Geralmente ela é determinada por uma série de exames e avaliações realizadas pelos médicos. Dessa forma, se os sintomas se encaixam em certos critérios, considerando o estado mental, incluindo avaliação cognitiva e outros exames físico, poderá haver indicação de Alzheimer.
Geralmente, este acomentimento vem acompanhado de alguma demência mental. Seja por um défict cognitivo ou outras limitações provocadas pela alteração de componentes do cérebro.
Sendo assim, com o objetivo de entendermos o perfil das pessoas que apresentam algum tipo de demência, vamos fazer uma análise descritiva somente com grupo de pessoas que possuem indicação de demência pelo teste neuropsicológico MMSE, classificados com algum grau de demência.

In [ ]:
# Verificando o quantitativo de pessoas com algum grau de classificação de demência.
ocs['CDR'].value_counts()

In [ ]:
# Criar novo dataframe somente com quem possui indicação de quadro de demência.
deme = ocs[(ocs['CDR'] > 0.0) & (ocs['MMSE'] <= 24)]

In [ ]:
# Visualizando as primeiras 5 linhas do novo dataframe.
deme.head(5)

In [ ]:
# Visualizando principais medidas deste grupo de pessoas.
deme.describe()

Os indivíduos que já apresentavam algum grau de demência estão no grupo de pessoas entre 63 e 92 anos.

Já no agrupamento realizado abaixo, observa-se que os maiores graus de classificação de demências estão atribuídos à pessoas com status sócio econômicos melhores.

In [ ]:
# Valores máximos de classificação de demência por Status Sócio Econômico.
ocs.groupby('SES')['CDR'].max().sort_values(ascending=False)

In [ ]:
# Valores máximos de classificação de demência por Grau de Instrução.
ocs.groupby('Educ')['CDR'].max().sort_values(ascending=False)

Gráfico indicando a correlação das variáveis analiadas no presente estudo.

In [ ]:
def plot_corr(corr):
    # Cortaremos a metade de cima pois é o espelho da metade de baixo
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask, 1)] = True
    sns.heatmap(corr, mask=mask, cmap='RdBu', square=True, linewidths=.5)
# Calculando a correlação
corr = ocs.corr() 
plot_corr(corr)

In [ ]:
sns.barplot(x='Educ' ,y='SES' ,data=ocs)

# Conclusão

Embora não haja como diagnosticar o Alzheimer, estes dados nos trouxeram algumas informações interessantes quanto a alguns fatores que podem ajudar a determinar a doença.

Percebemos que a idade, de fato, pode inibir algumas funções cognitivas. Percepção proveniente da diminuição do volume intracraneano e alteração de testes neuropsicológicos com indicação de algum tipo de demência. Quando comparado a ocorrência destas variáveis a nível de gêneros masculinos e femeninos, a redução do volume intracraneado é mais recorrente em mulheres que em homens. Entretanto, o desempenho de exames neuropsicológicos são mais baixos entre os homens que nas mulheres. 

Não foi encontrada nenhuma relação siginificativa com o grau de instrução do indivíduo ou status sócio econômico que indicasse prejuízo nas variáveis de estados mentais neste teste.

Concluímos assim, que a idade e o sexo podem trazer circunstâncias que influenciam em fatores que determinam o Alzheimer, como o volume e comportamento do cérebro. Mas ainda assim, são insuficientes para determinar a doença. Esta pesquisa precisaria abranger outros pontos relevantes na determinação da doença Alzheimer. Como já mencionado neste estudo, seria necessário agregar outros tipos de avaliação, como histórico pessoal, exames físicos, testes laboratoriais e demais exames neurológicos.
